In [1]:
import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.append("../federated")


from notears.utils import set_random_seed, simulate_dag, simulate_parameter, simulate_linear_sem
from notears.linear import notears_linear, notears_linear_fed

from utils import calculate_metrics

set_random_seed(1)

## Notears

#### Dataset generation

In [16]:
n, d, s0, graph_type, sem_type = 5000, 20, 40, 'ER', 'gauss'

adj_mat_true = simulate_parameter(simulate_dag(d, s0, graph_type))

dataset = simulate_linear_sem(adj_mat_true, n, sem_type)
print(f'Shape of dataset: {dataset.shape}')
print(f'Adjacency matrix: {adj_mat_true}')

Shape of dataset: (5000, 20)
Adjacency matrix: [[ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.         -1.03343447  0.
   0.          0.50541388  0.         -0.67660725  0.          0.
   0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.        ]
 [ 0.         -1.13215183  0.          0.          0.          0.
   0.          0.         -1.64029786  0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.        ]
 [ 0.          0.67867058  1.44043579  0.          0.

#### Without belief aggregation

In [17]:
last_w_est = notears_linear(dataset, lambda1=0.01, loss_type='l2', ground_truth=adj_mat_true)

> 0 SHD = 40
> 1 SHD = 17
> 2 SHD = 5
> 3 SHD = 3
> 4 SHD = 0
> 5 SHD = 2
> 6 SHD = 0
> 7 SHD = 0
> 8 SHD = 0
> 9 SHD = 0
> 10 SHD = 0
> 11 SHD = 0


#### With belief aggregation

In [ ]:
prior = notears_linear_fed(dataset, W_prior=last_w_est, lambda1=0.01, loss_type='l2', ground_truth=adj_mat_true)

> 0 SHD = 40
> 1 SHD = 17
> 2 SHD = 6


In [ ]:
prior = notears_linear_fed(dataset, W_prior=np.ones(800), lambda1=0.1, loss_type='l2', ground_truth=adj_mat_true)

In [ ]:
prior = notears_linear_fed(dataset, W_prior=np.zeros(800), lambda1=0.1, loss_type='l2', ground_truth=adj_mat_true)

## IGSP and GIES


In [3]:
import os
import sys
sys.path.append("../federated")

import numpy as np
import matplotlib.pyplot as plt

from logging_settings import logger

### Set experiments configurations

In [2]:
# Graph
graph_types = ["full", "bidiag", "chain", "bidiag", "jungle"]
edge_probs = [0.1, 0.2, 0.4, 0.6, 0.8, 1.0]
num_vars = 20

# Dataset
obs_data_size = 5000
int_data_size = num_vars * 2

# Method
gies_method = "gies_experiment.py"
igsp_method = "igsp_experiment.py"
methods = [gies_method, igsp_method]

# Federated
num_clients = 2
num_rounds = 1
num_seeds = 5

### Sample Run


In [5]:
command = f'python {gies_method} --obs-samples-size {obs_data_size} --int-samples-size {int_data_size}' \
            f'--num-vars {num_vars} --graph-type {"random"} --edge-prob {0.0} --seed {0}' \
            f'--num-clients {num_clients} --num-rounds {num_rounds}'
logger.info(f'Executing command: {command}')

os.system(command)

INFO - <ipython-input-5-d4e6afb87e13> - 4 - 2021-11-29 13:11:52,859 - Executing command: python gies_experiment.py --obs-samples-size 5000 --int-samples-size 40--num-vars 20 --graph-type random --edge-prob 0.0 --seed 0--num-clients 2 --num-rounds 1


512

### Structured graphs

In [3]:
edge_prob = 0.0

for method in methods: 
    for graph_type in graph_types:
        for seed in range(num_seeds):
            command = f'python {method} --obs-samples-size {obs_data_size} --int-samples-size {int_data_size}' \
                      f'--num-vars {num_vars} --graph-type {graph_type} --edge-prob {edge_prob} --seed {seed}' \
                      f'--num-clients {num_clients} --num-rounds {num_rounds}'
            logger.info(f'Executing command: {command}')

            os.system(command)


INFO - <ipython-input-3-b2fad49b0164> - 9 - 2021-11-29 13:03:45,755 - Executing command: python gies_experiment.py --obs-samples-size 5000 --int-samples-size 40--num-vars 20 --graph-type full --edge-prob 0.0 --seed 0--num-clients 2 --num-rounds 1
INFO - <ipython-input-3-b2fad49b0164> - 9 - 2021-11-29 13:03:53,164 - Executing command: python gies_experiment.py --obs-samples-size 5000 --int-samples-size 40--num-vars 20 --graph-type full --edge-prob 0.0 --seed 1--num-clients 2 --num-rounds 1
INFO - <ipython-input-3-b2fad49b0164> - 9 - 2021-11-29 13:03:54,921 - Executing command: python gies_experiment.py --obs-samples-size 5000 --int-samples-size 40--num-vars 20 --graph-type full --edge-prob 0.0 --seed 2--num-clients 2 --num-rounds 1
INFO - <ipython-input-3-b2fad49b0164> - 9 - 2021-11-29 13:03:56,856 - Executing command: python gies_experiment.py --obs-samples-size 5000 --int-samples-size 40--num-vars 20 --graph-type full --edge-prob 0.0 --seed 3--num-clients 2 --num-rounds 1
INFO - <ipyt

### Random graphs

In [ ]:
graph_type = "random"
for edge_prob in edge_probs:
    command = f'python {method} --obs-samples-size {obs_data_size} --int-samples-size {int_data_size}' \
            f'--num-vars {num_vars} --graph-type {graph_type} --edge-prob {edge_prob} --seed {seed}' \
            f'--num-clients {num_clients} --num-rounds {num_rounds}'